# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [2]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

#Extract Data from File
nhl_df=pd.read_csv("assets/nhl.csv")
nhl_df['team']=nhl_df['team'].str.replace(r'\*',"")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

#Clear Data (Population)
cities=cities[['Metropolitan area',"Population (2016 est.)[8]",'NHL']]
cities.columns=['Metropolitan area',"Population",'NFL']
cities["NFL"]=cities["NFL"].str.replace(r'\[.*\]','')
cities=cities[cities['NFL'].str.match('\w.*')]

#Separate two team in one column
cities['NFL']=cities['NFL'].map(lambda s: [t.strip() for t in re.split("([A-Z][^A-Z]*)",s)][1::2])
cities=cities.explode('NFL')

#Rename Columns
cities.columns=['Metropolitan area',"Population",'team']
cities=cities.reset_index(drop=True)

#Clear Data (Ratio)
nhl_df=nhl_df[nhl_df['year']==2018]
nhl_df=nhl_df[nhl_df['W'].str.match('\d.*')]
nhl_df['W']=nhl_df['W'].map(lambda s: int(s) if s.isdigit() else np.nan)
nhl_df['L']=nhl_df['L'].map(lambda s: int(s))
nhl_df=nhl_df[['team','W',"L"]]

#Calculate the Win Rate
nhl_df['Ratio']=nhl_df['W']/(nhl_df['W']+nhl_df['L'])
nhl_df['team']=nhl_df['team'].str.replace(r'(.*) ',"")

#Combine two group of Data
cities=pd.merge(cities,nhl_df,how='outer',on='team')
cities['Population']=cities['Population'].map(lambda x:int(x))
cities=cities.groupby('Metropolitan area').agg('mean')

def nhl_correlation(): 
    population_by_region = cities['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = cities['Ratio'] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

nhl_correlation()

0.012486162921209907

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [3]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

#Clean Data (Ratio)
nba_df=pd.read_csv("assets/nba.csv")
nba_df=nba_df.set_index('team')
nba_df=nba_df[nba_df['year']==2018]
nba_df.index=nba_df.index.str.replace(r'\(.*\)','')
nba_df.index=nba_df.index.str.replace(r'\*','')
nba_df.index=nba_df.index.str.replace(r'(.*) ','')
nba_df=nba_df.reset_index()
nba_df=nba_df[['team','W','L','W/L%']]
nba_df['W']=nba_df['W'].map(lambda x:int(x))
nba_df['L']=nba_df['L'].map(lambda x:int(x))
nba_df['team']=nba_df['team'].map(lambda x:x.strip())
nba_df.columns=['team','W','L','Ratio']
#print(nba_df)

#Clean Data (Population)
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
cities['NBA']=cities['NBA'].str.replace(r'\[.*\]','')
cities=cities[['Metropolitan area',"Population (2016 est.)[8]",'NBA']]
cities.columns=['Metropolitan area',"Population",'NBA']
cities=cities[cities['NBA'].str.match('\w.*')]
cities['NBA']=cities['NBA'].map(lambda s: [t.strip() for t in re.split("([A-Z0-9][^A-Z]*)",s)][1::2])
cities=cities.explode('NBA')
cities.columns=['Metropolitan area',"Population",'team']
cities['Population']=cities['Population'].map(lambda x:int(x))
cities=cities.reset_index(drop=True)
#print(cities)

#Merge
cities=pd.merge(cities,nba_df,how='outer',on='team')
cities=cities.groupby('Metropolitan area').agg('mean')
cities['Ratio']=cities['W']/(cities['W']+cities['L'])
#print(cities)

def nba_correlation():
    population_by_region = cities['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = cities['Ratio'] # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

nba_correlation()

-0.17657160252844617

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [5]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

#Load Data
mlb_df=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
cities=cities[['Metropolitan area','Population (2016 est.)[8]','MLB']]
cities['MLB']=cities['MLB'].str.replace('\[.*\]','')
cities=cities[cities['MLB'].str.match('\w.*')]
cities['MLB']=cities['MLB'].str.replace(' S','s')
cities['MLB']=cities['MLB'].str.replace(' J','j')
cities['MLB']=cities['MLB'].map(lambda s: [t.strip() for t in re.split("([A-Z0-9][^A-Z]*)",s)][1::2])
cities=cities.explode('MLB')
cities.columns=['Metropolitan area','Population','team']
cities['Population']=cities['Population'].map(lambda x:int(x))
cities=cities.reset_index(drop=True)
#print(cities)

#Clean Data ()
mlb_df=mlb_df[mlb_df['year']==2018]
mlb_df=mlb_df[['team','W','L','W-L%']]
mlb_df['team']=mlb_df['team'].str.replace(' S','s')
mlb_df['team']=mlb_df['team'].str.replace(' J','j')
mlb_df['team']=mlb_df['team'].str.replace('(.*) ','')
#print(mlb_df)

#Merge
cities=pd.merge(cities,mlb_df,on='team')
cities=cities.groupby('Metropolitan area').mean()

def mlb_correlation(): 
   
    population_by_region = cities['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = cities['W-L%'] # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

mlb_correlation()

0.15003737475409495

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [6]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

#load Data
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
cities=cities[['Metropolitan area', 'Population (2016 est.)[8]','NFL']]
cities['NFL']=cities['NFL'].str.replace(r'\[.*\]',"")
cities=cities[cities['NFL'].str.match('\w.*')]
cities['NFL']=cities['NFL'].map(lambda s: [t.strip() for t in re.split("([A-Z0-9][^A-Z]*)",s)][1::2])
cities=cities.explode('NFL')
cities.columns=['Metropolitan area', 'Population','team']
cities['Population']=cities['Population'].map(lambda x:int(x))
#print(cities)

#Clean Data
nfl_df=nfl_df[nfl_df['year']==2018]
nfl_df=nfl_df[['team','W-L%']]
nfl_df['team']=nfl_df['team'].str.replace('[*+]','')
nfl_df['W-L%']=(nfl_df['W-L%'].str.replace('[A-Z]* \w*','0')).map(lambda x:float('0'+x))
nfl_df=nfl_df[nfl_df['W-L%']>0]
nfl_df['team']=nfl_df['team'].str.replace(r'(.*) ','')
#print(nfl_df)

#Merge
cities=pd.merge(cities,nfl_df,on='team')
cities=cities.groupby('Metropolitan area').agg('mean')
#print(cities)
def nfl_correlation(): 
    
    population_by_region = cities['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = cities['W-L%'] # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

nfl_correlation()

0.004282141436393017

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re
'''
mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
'''
def sports_team_performance():
        
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values